In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
%cd /content/gdrive/MyDrive/llava_finalproject/llava_finalproject
!pip install transformers torch accelerate pillow

/content/gdrive/.shortcut-targets-by-id/1Mbt85qT9H1kdK75BurfXdSWxchPRLt3H/llava_finalproject


In [ ]:
!pip install kagglehub

In [ ]:
from transformers import AutoConfig
import numpy as np
import torch
from torch.utils.data import DataLoader, Subset
from vicuna_llava import vicuna_llava, dataset_llava
from accelerate import Accelerator
accelerator = Accelerator()

In [ ]:
from huggingface_hub import login
hf_token = ""
login(token=hf_token)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# grab vicuna and its tokenizer
model_name = "meta-llama/Llama-3.2-1B"#"lmsys/vicuna-7b-v1.5"
config = AutoConfig.from_pretrained(model_name)

vicunallava = accelerator.prepare(vicuna_llava(config, llmURL=model_name, accelerator=accelerator))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/843 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/1.71G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


In [ ]:
linear_llava_proj = torch.load('Pretrained/vicunallava_im_embedding_stage1.pt', map_location=device)

with torch.no_grad():
    vicunallava.im_embedding.weight.copy_(linear_llava_proj.weight)
    vicunallava.im_embedding.bias.copy_(linear_llava_proj.bias)

<ipython-input-7-b7b4949d37d0>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  linear_llava_proj = torch.load('Pretrained/vicunallava_im_embedding_stage1.pt', map_location=

In [ ]:
!pip install --force-reinstall -v openai==1.55.3

Using pip 24.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
  Obtaining dependency information for openai==1.55.3 from https://files.pythonhosted.org/packages/10/06/691ef3f0112ecf0d7420d0bf35b5d16cf81554141f4b4913a9831031013d/openai-1.55.3-py3-none-any.whl.metadata
  Obtaining dependency information for anyio<5,>=3.5.0 from https://files.pythonhosted.org/packages/a0/7a/4daaf3b6c08ad7ceffea4634ec206faeff697526421c20f07628c7372156/anyio-4.7.0-py3-none-any.whl.metadata
  Obtaining dependency information for distro<2,>=1.7.0 from https://files.pythonhosted.org/packages/12/b3/231ffd4ab1fc9d679809f356cebee130ac7daa00d6d6f3206dd4fd137e9e/distro-1.9.0-py3-none-any.whl.metadata
  Obtaining dependency information for httpx<1,>=0.23.0 from https://files.pythonhosted.org/packages/2a/39/e50c7c3a983047577ee07d2a9e53faf5a69493943ec3f6a384bdc792deb2/httpx-0.28.1-py3-none-any.whl.metadata
  Obtaining dependency information for jiter<1,>=0.4.0 from https://files.pythonhosted.org/pack

In [ ]:
%ls /content/gdrive/MyDrive/llava_finalproject/Aerial_Landscapes/River

ls: cannot access '/content/gdrive/MyDrive/llava_finalproject/Aerial_Landscapes/River': No such file or directory


In [ ]:
import base64
import os
from openai import OpenAI
# import openai
import json
from skyview_dataset import skyview_data

api_key = ""
client = OpenAI(api_key=api_key)

def encode_image(folder, image_filename):
    image_path = os.path.join("/content/gdrive/MyDrive/llava_finalproject/llava_finalproject/Aerial_Landscapes", folder, image_filename)
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

# The user wants to understand whether there is a place in the image that someone could land a small airplane. So being descriptive with the image given this context is crucial.
#            If there is no possible emergency landing areas then this is also something to consider. Keep your output to 5 sentences, with the last sentence giving a score on feasibility of landing an airplane. Specifically, the first four sentences will describe the landscape and the pros and cons of it given this emergency landing scenario.
#            The last sentence you should give a score out of 100 for whether the plane can land somewhere in the image or not, and the chance of survival if landed. 0 would indicate there is no survival chances, such as in mountainous areas or deep oceans and 100 would indicate an airport which is really safe to land on.
            # Note: a small airplane is 26 feet long with a wingspan of 36 feet and a cabin height of roughly 4 feet. Here is an example response:
            # 1) The landscape shows a winding river surrounded by uneven terrain, with visible vegetation and some open areas.
            # 2) The riverbanks appear to have patches of flat land, but the irregularity and potential wetness of the soil pose risks.
            # 3) Vegetation is scattered, which could obstruct a small plane's landing, and the lack of a consistent open stretch limits options for a safe touchdown.
            # 4) While the river itself is unsuitable for landing, the narrow flat stretches near it may offer limited possibilities in an extreme emergency, though precision is critical.
            # Score: 35/100 - The terrain presents significant challenges for landing, with survival chances depending on precision and the condition of the nearby flat areas.

system_message = {
    "role": "system",
    "content": [
        {
        "type": "text",
        "text": """You are a helpful vision assistant that is going to help me understand the terrains of landscape images. The user wants to know if there is an area where a plane could land within the image."""
        }
    ]
}

llava_message = """Can a plane safely land in this area?"""
# system_message['content'][0]["text"] = llava_message
print(system_message)

{'role': 'system', 'content': [{'type': 'text', 'text': 'You are a helpful vision assistant that is going to help me understand the terrains of landscape images. The user wants to know if there is an area where a plane could land within the image.'}]}


In [ ]:

for datum in skyview_data:
    base64_image = encode_image(datum["folder"], datum["image"])

    user_content = [
        {
            "type": "image_url",
            "image_url": {
                "url": f"data:image/jpeg;base64,{base64_image}"
            }
        }
    ]

    # Make the API call
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            system_message,
            {
                "role": "user",
                "content": user_content
            }
        ],
    )

    answer = response.choices[0].message.content

    datum["gpt"] = answer

    # Make the API call
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            system_message,
            {
                "role": "user",
                "content": "Write a random description about whether it is safe to land a plane at given area or not."
            }
        ],
    )
    datum["baseline"] = response.choices[0].message.content

with open("skyview_data_gpt.json", "w") as outfile:
    json.dump(skyview_data, outfile, indent=2)

# #TEST
# for d in skyview_data[:5]:
#     print(f"ID: {d['id']}, Folder: {d['folder']}, Image: {d['image']}")
#     print("GPT Analysis:", d["gpt"])
#     print("---")


In [ ]:
%ls

Aerial_Landscapes/    Pretrained/        scottyh.jpg            Stage2_Checkpoint/
eval.ipynb            pretraining.ipynb  skyview_data_gpt.json  temp.py
llava_init_wip.ipynb  __pycache__/       skyview_dataset.py     vicuna_llava.py


In [ ]:
llamallava = None

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# grab llama
model_name = "meta-llama/Llama-3.2-1B"#"lmsys/vicuna-7b-v1.5"
#config = AutoConfig.from_pretrained(model_name)
# Grab config from the stage 2 pretrain folder
config = AutoConfig.from_pretrained("Stage2_Checkpoint")

# instantiate the model like normal
llamallava = accelerator.prepare(vicuna_llava(config, llmURL=model_name, accelerator=accelerator))
state_dict = torch.load("/content/gdrive/MyDrive/llava_finalproject/llava_finalproject/Pretrained/llamallava_stage2_epoch1.pt")
llamallava.load_state_dict(state_dict, strict=False)

# load the state_dict using the .safetensors from model checkpoint
# from safetensors.torch import load_file
# state_dict_part1 = load_file("/content/gdrive/MyDrive/llava_finalproject/Stage2_Checkpoint/model-00001-of-00002.safetensors")
# state_dict_part2 = load_file("/content/gdrive/MyDrive/llava_finalproject/Stage2_Checkpoint/model-00002-of-00002.safetensors")
# state_dict = {**state_dict_part1, **state_dict_part2}
# state_dict = {k: v for k, v in state_dict.items() if k != "model.lm_head.weight"}

# load state_dict into the llamallava model
# llamallava.load_state_dict(state_dict, strict=False)

<ipython-input-11-86553d5bc16c>:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load("/content/gdrive/MyDrive/llava_finalproject/llava_finalproject/Pret

<All keys matched successfully>

In [ ]:
# CALL LLAVA AND ASK GPT TO COMPARE SCORES AND STORE
from PIL import Image
import json

data = []
with open("skyview_data_gpt.json", "r") as infile:
    data = json.load(infile)


new_system_message = {
    "role": "system",
    "content": [
        {
        "type": "text",
        "text": """ You are going to help me judge the following responses with regard to the accuracy of their statements. Specifically, your job is to rate the descriptions based on their conclusions about landing a plane.
                    You should prioritize correct answers over detailed responses. Responses with details outside of what can be observed in the image, such as weather conditions, should receive a poor score.
                    Additionally, responses considering factors outside of the immediate context in the image such as legality should also recieve a poor score.
                    For each pair of responses grade their reasoning out of 10. Your response should be exactly as follows.
                    Respond with a JSON object. Ensure the first character of the output is '{' and the last character is '}'.
                    The JSON format you will use is {"REASONING": <your reasoning here>, "RESPONSE 1 SCORE": <number 0-10>, "RESPONSE 2 SCORE": <number 0-10>, "RESPONSE 3 SCORE": <number 0-10>}.
                    Do not reply with anything other than this JSON object.
                    For example:
                    {"REASONING": 'The reasoning for this evaluation is based on X, Y, and Z.', "RESPONSE 1 SCORE": 8, "RESPONSE 2 SCORE": 9, "RESPONSE 3 SCORE": 7}"""
        }

    ]
}

scores = {'id': [], 'llava': [], 'gpt': [], 'baseline': [], 'unable to evaluate': []} # STORE SCORES LIKE GPT-> [] LLAVA > []

for datum in data:
    base64_image = encode_image(datum["folder"], datum["image"])

    # CALL LLAVA HERE
    im = Image.open(os.path.join("/content/gdrive/MyDrive/llava_finalproject/llava_finalproject/Aerial_Landscapes", datum["folder"], datum["image"]))
    llava_res = llamallava.generate(im, llava_message, max_new_tokens=200)
    print(llava_res)


    user_content = [

        {
            "type": "text",
            "text": f"""
            RESPONSE 1: {llava_res},
            RESPONSE 2: {datum['gpt']},
            RESPONSE 3: {datum['baseline']}"""

        },
        {
            "type": "image_url",
            "image_url": {
                "url": f"data:image/jpeg;base64,{base64_image}"
            }
        }
    ]

    # Make the API call
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            new_system_message,
            {
                "role": "user",
                "content": user_content
            }
        ],
    )
    print(response)

    res = response.choices[0].message.content



    try:
      if res.startswith("'''json\n"):
        res = input_string[len("json\n"):-3]
      res = json.loads(res)
      gpt_score = res['RESPONSE 2 SCORE']
      llava_score = res['RESPONSE 1 SCORE']
      baseline = res['RESPONSE 3 SCORE']

      scores['id'] += [datum['id']]
      scores['gpt'] += [gpt_score]
      scores['llava'] += [llava_score]
      scores['baseline'] += [baseline]
    except:
      scores['unable to evaluate'] = datum['id']


    #PARSE RESPONSE, MAKE SURE GPT RESPONDS IN A WAY THAT IS EASIER TO RESPOND



with open("skyview_eval_scores.json", "w") as outfile:
    json.dump(scores, outfile, indent=2)

yes, the area is suitable for landing a plane. The land is flat and the weather is clear, so the plane can land safely in the area.<|end_of_text|>
ChatCompletion(id='chatcmpl-AeSVgA2M0HAaUXczMSPPIz47kc7PM', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='{"REASONING": "The evaluation focuses on how each response addresses the visible terrain features in the image for landing suitability. Response 1 talks about clear weather and flat land, but adds an assumption about weather, which wasn’t asked for. Response 2 accurately describes the terrain features and raises valid concerns about the suitability given the presence of waterways and elevations. Response 3 also effectively analyzes the terrain but introduces non-relevant factors like legality which should not be considered.", "RESPONSE 1 SCORE": 5, "RESPONSE 2 SCORE": 8, "RESPONSE 3 SCORE": 7}', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], creat